In [59]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,accuracy_score
import joblib

In [61]:
#Load Dataset
df=pd.read_csv("data_to_be_cleansed.csv")
#Remove redundant columns
df=df.iloc[::,1:]
#Drop rows with missing 'text' values
df=df.dropna(subset=['text'])
#Remove Duplicates
df=df.drop_duplicates(subset=['text','title'])
# Combine 'text' and 'title' columns
df['input'] = df['text'] + " " + df['title']

In [63]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and numbers
    text = re.sub(r'[^a-z\s]', '', text)
    # Tokenize and remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

# Apply preprocessing to 'text' and 'title' columns
df['input_text'] = df['input'].apply(preprocess_text)
# Drop the original 'text' and 'title' columns if not needed
df.drop(columns=['text', 'title', 'input'], inplace=True)
# Save the cleaned dataset
df.to_csv('clean.csv', index=False)

In [65]:
# Load the cleaned dataset
df1=pd.read_csv('clean.csv')
# Initialize TF-IDF vectorizer
# vz=TfidfVectorizer(max_features=5000)  # Adjust max_features as needed
# Extract features from processed text
cv=CountVectorizer(max_features=5000,stop_words='english')
X=cv.fit_transform(df1['input_text']).toarray()
# Target column
y=df1['target']

In [67]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=1,test_size=0.3)
model=LogisticRegression(max_iter=1000,random_state=1)
model.fit(X_train,y_train)
pred_train=model.predict(X_train)
print('Training Score:',accuracy_score(pred_train,y_train))
pred_test=model.predict(X_test)
print('Testing Score:',accuracy_score(pred_test,y_test))

Training Score: 0.9958064516129033
Testing Score: 0.6975169300225733


In [71]:
# Example user input
new_text = [input()]
 # Replace with your text

# Preprocess the input (vectorize using the same TF-IDF vectorizer)
new_text_vectorized = cv.transform(new_text).toarray()

# Predict the class
predicted_class = model.predict(new_text_vectorized)

# Get the probability of each class
predicted_probabilities = model.predict_proba(new_text_vectorized)

# Display the results
print("Predicted Class:", predicted_class[0])
print("Class Probabilities:", predicted_probabilities[0])
dic={0:'Stress',1:'Depression',2:'Bipolar disorder',3:'Personality disorder',4:'Anxiety'}
result=dic[predicted_class[0]]
new_text=f'I am suffering from {result}'

 stress


Predicted Class: 0
Class Probabilities: [0.86142815 0.053794   0.01236425 0.04557896 0.02683463]


In [73]:
dump=joblib.dump(model,'model.pkl')
dump2=joblib.dump(cv,'cv.joblib')